#### Limpeza Deputados

In [17]:
import pandas as pd
import os

In [18]:
data = pd.read_csv('../Coleta/dados-coletados/deputados.csv')
data.head()

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,139356,https://dadosabertos.camara.leg.br/api/v2/depu...,ROSÁRIO CONGRO NETO,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48,https://www.camara.leg.br/internet/deputado/ba...,NaN
1,1507,https://dadosabertos.camara.leg.br/api/v2/depu...,FADAH GATTASS,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48,https://www.camara.leg.br/internet/deputado/ba...,NaN
2,1509,https://dadosabertos.camara.leg.br/api/v2/depu...,JUAREZ MARQUES BATISTA,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48,https://www.camara.leg.br/internet/deputado/ba...,NaN
3,1516,https://dadosabertos.camara.leg.br/api/v2/depu...,ABDALLA JALLAD,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48,https://www.camara.leg.br/internet/deputado/ba...,NaN
4,139284,https://dadosabertos.camara.leg.br/api/v2/depu...,LEVY DIAS,PFL,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48,https://www.camara.leg.br/internet/deputado/ba...,NaN


In [19]:
# Exclui as colunas com dados desnecessários
data.drop(['uri', 'urlFoto', 'email'], axis=1, inplace=True)
data.head()

,id,nome,siglaPartido,uriPartido,siglaUf,idLegislatura
0,139356,ROSÁRIO CONGRO NETO,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48
1,1507,FADAH GATTASS,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48
2,1509,JUAREZ MARQUES BATISTA,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48
3,1516,ABDALLA JALLAD,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48
4,139284,LEVY DIAS,PFL,https://dadosabertos.camara.leg.br/api/v2/part...,MS,48


In [20]:
# Exclui todos os deputados anteriores à 52ª legislatura, que é a partir de quando existem dados relevantes das votações

legistatura_min = 52

data = data[data['idLegislatura'] >= legistatura_min]
data.head()

,id,nome,siglaPartido,uriPartido,siglaUf,idLegislatura
2486,74240,JOÃO TOTA,PP**,https://dadosabertos.camara.leg.br/api/v2/part...,AC,52
2487,74184,RONIVON SANTIAGO,PP**,https://dadosabertos.camara.leg.br/api/v2/part...,AC,52
2488,73499,NARCISO MENDES,PP**,https://dadosabertos.camara.leg.br/api/v2/part...,AC,52
2489,74181,CHICÃO BRÍGIDO,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,AC,52
2490,74365,NILSON MOURÃO,PT,https://dadosabertos.camara.leg.br/api/v2/part...,AC,52


In [21]:
# Extrai o id do partido do link do mesmo e depois exclui a coluna com os links dos partidos
data['idPartido'] = data['uriPartido'].str.extract('(\d+)$')
data.drop('uriPartido', axis=1, inplace=True)

cols = data.columns.tolist()
cols.insert(3, cols.pop(cols.index('idPartido')))
data = data[cols]

data['idPartido'] = data['idPartido'].astype('int')

data.head()

,id,nome,siglaPartido,idPartido,siglaUf,idLegislatura
2486,74240,JOÃO TOTA,PP**,36809,AC,52
2487,74184,RONIVON SANTIAGO,PP**,36809,AC,52
2488,73499,NARCISO MENDES,PP**,36809,AC,52
2489,74181,CHICÃO BRÍGIDO,PMDB,36800,AC,52
2490,74365,NILSON MOURÃO,PT,36844,AC,52


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3763 entries, 2486 to 6248
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             3763 non-null   int64 
 1   nome           3763 non-null   object
 2   siglaPartido   3763 non-null   object
 3   idPartido      3763 non-null   int32 
 4   siglaUf        3763 non-null   object
 5   idLegislatura  3763 non-null   int64 
dtypes: int32(1), int64(2), object(3)
memory usage: 191.1+ KB


In [23]:
data.to_csv('./dados-limpos/deputados.csv', index=False)

#### Limpeza Votações

In [24]:
data_sample = pd.read_csv('../Coleta/dados-coletados/votos-2003.csv', sep=';')
data_sample.head()

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto
0,104509-28,https://dadosabertos.camara.leg.br/api/v2/vota...,2003-11-18T16:25:57,Não,73928,https://dadosabertos.camara.leg.br/api/v2/depu...,Ann Pontes,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,PA,52,https://www.camara.leg.br/internet/deputado/ba...
1,104509-28,https://dadosabertos.camara.leg.br/api/v2/vota...,2003-11-18T16:25:57,Não,73434,https://dadosabertos.camara.leg.br/api/v2/depu...,Arnaldo Faria de Sá,PTB,https://dadosabertos.camara.leg.br/api/v2/part...,SP,52,https://www.camara.leg.br/internet/deputado/ba...
2,104509-28,https://dadosabertos.camara.leg.br/api/v2/vota...,2003-11-18T16:25:57,Sim,74263,https://dadosabertos.camara.leg.br/api/v2/depu...,Cláudio Magrão,PPS,https://dadosabertos.camara.leg.br/api/v2/part...,SP,52,https://www.camara.leg.br/internet/deputado/ba...
3,104509-28,https://dadosabertos.camara.leg.br/api/v2/vota...,2003-11-18T16:25:57,Sim,74060,https://dadosabertos.camara.leg.br/api/v2/depu...,Daniel Almeida,PCdoB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,52,https://www.camara.leg.br/internet/deputado/ba...
4,104509-28,https://dadosabertos.camara.leg.br/api/v2/vota...,2003-11-18T16:25:57,Sim,74381,https://dadosabertos.camara.leg.br/api/v2/depu...,Dra. Clair,PT,https://dadosabertos.camara.leg.br/api/v2/part...,PR,52,https://www.camara.leg.br/internet/deputado/ba...


In [25]:
data_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12380 entries, 0 to 12379
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   idVotacao               12380 non-null  object
 1   uriVotacao              12380 non-null  object
 2   dataHoraVoto            12380 non-null  object
 3   voto                    12380 non-null  object
 4   deputado_id             12380 non-null  int64 
 5   deputado_uri            12380 non-null  object
 6   deputado_nome           12380 non-null  object
 7   deputado_siglaPartido   12380 non-null  object
 8   deputado_uriPartido     12380 non-null  object
 9   deputado_siglaUf        12380 non-null  object
 10  deputado_idLegislatura  12380 non-null  int64 
 11  deputado_urlFoto        12380 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.1+ MB


In [10]:
# Limpeza dos dados de votações
def clean_data(year):
    data = pd.read_csv(f'../Coleta/dados-coletados/votos-{year}.csv', sep=';')
    
    # Encontra o id do partido no link do mesmo
    data['idPartido'] = data['deputado_uriPartido'].str.extract('(\d+)$')

    cols = data.columns.tolist()
    cols.insert(8, cols.pop(cols.index('idPartido')))
    data = data[cols]
    
    # Exclui colunas desnecessárias
    data.drop(['uriVotacao', 'deputado_uri', 'deputado_urlFoto', 'deputado_uriPartido'], axis=1, inplace=True)
   
    # Em alguns casos a coluna 'voto' é nula indicando que o voto não foi realizado. O mesmo ocorre em outras colunas
    # Todos os casos em que uma das colunas é nula não fazem sentido para o escopo desse trabalho, logo serão removidos
    data = data.dropna()

    data['idPartido'] = data['idPartido'].astype('int')

    data.to_csv(f'./dados-limpos/votos-{year}.csv', index=False)

In [11]:
for year in range(2003, 2024):
    clean_data(year)

In [13]:
# Concatena todos os arquivos com dados de votações em um único arquivo e exclui os originais
dataframes = []

for year in range(2003, 2024):
    df = pd.read_csv(f'./dados-limpos/votos-{year}.csv')
    dataframes.append(df)

all_data = pd.concat(dataframes, ignore_index=True)

all_data.to_csv(f'./dados-limpos/votos.csv', index=False)

for year in range(2003, 2024):
    os.remove(f'./dados-limpos/votos-{year}.csv')